In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator
from collections import Counter

## Отчет по маркетинговой эффективности
Необходимо обновить отчет за неделю - недели, последняя актуальная версия должна лежать здесь

\tsclient\X\УМиР\2021\аналитика\отчет для руководства

Название "ОСводный отчет по оценке эффективности маркетинговых активностей"

1) На текущий момент для того чтобы сделать отчет необходимо производить ручные выгрузки из Маркетинг --> Отчет по продажам

Делаем по сумме чеков, доходу, кол-ву чеков.

Т.е. понадобятся следующие выгрузки

40-300

300-2950

2950-3000

Файлы называем также и кладем в соответствующую папку:

Доход

Оборот

Чеки


Делаем это для показателей как по неделе, так и только по выходным дням.

Называю по типу Sales_11042021_all; Sales_11042021_holidays

2) количество клиентов, предоставивших документы строка 30 - клиенты из RFM с суммой от 3100000р, следующая строка 31 - кол-во клиентов с документами, получивших скидку, брать из отчета по БанкиРу "Ничоси"

3) Выгрузить RFM за нужный период

4) Выгрузить ВОО за нужный период

5) Выгрузить ВОО за пол года до нужного периода (в названии при сохранении указать half_year)

6) Строки 44 и 45, Программы лояльности --> Отчет по операциям по карте лояльности (Для аналитики), выбрать нужный период. Брать нужно значение из столбца N, не забыть удалить операции из других периодов).

7) Строка 50 (кол-во активных участников клуба) Интерфейс Личный кабинет --> Отчеты --> Статистика использования мобильного приложения Заубер Club. Выбираем "Ежеденевная активность" за последние 30 дней. Берем max за неделю. 

Сохранил файл как "Статистика_исп_моб_приложения"

8) Кол-во новых участников клуба

Интерфейс Программа лояльности --> Документы --> Список карт. Выгрузить и сохранить как Список_карт  Для заполнения строки 49

9) Кол-во скачиваний приложения. Строка 48. 

Интерфейс Личный кабинет --> Отчеты --> Статистика использования мобильного приложения Заубер Club --> Установки и удаления по месяцам --> Сформировать.

Вписать необходимо в первую ячейку месяца.



In [2]:
FOLDER = r'D:\FTP-DATA\Analytics\Marketing_effiquency\23052021'

DATE_PERIOD = '17.05.2021-23.05.2021'

year = 2021

dates = list(map(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y'), DATE_PERIOD.split('-')))

os.chdir(FOLDER)

## Определим папки из которых будем брать соотвествующие показатели для строк 4-23 и 51-70 и обработаем необходимые файлы

In [3]:
sales_folders = [element for element in os.listdir() if os.path.isdir(element) and 'sales' in element.lower()]

In [4]:
indicators = pd.ExcelWriter('Показатели_банковские.xlsx')

In [5]:



intervals = {'40-300': {'Крупные': 'Удалить', 'Средние': '40 тыс-300тыс', 'Мелкие': 'до 40 тыс', 'Итого оборот:': 'Удалить'},
            '300-2950': {'Крупные': 'Удалить', 'Средние': '300 тыс-2950 тыс', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'},
            '2950-3000': {'Крупные': '3 млн более', 'Средние': '2950 тыс - 3 млн', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'}}

def sort_func(arr):
    arr = arr.to_list()
    
    dct = {'до 40 тыс': 1,
'40 тыс-300тыс': 2,
'300 тыс-2950 тыс': 3,
'2950 тыс - 3 млн': 4,
'3 млн более': 5}
    
    dct2 = {'Чеки': 1, 'Оборот': 2, 'Доход': 3}
    
    if 'Доход' in arr:
        return [dct2[i] for i in arr]
    else:
        return [dct[i] for i in arr]


for sale_folder in sales_folders:
    folders = [element for element in os.listdir(f'{sale_folder}') if os.path.isdir(f'{sale_folder}\\{element}')]
    temp = pd.DataFrame()
    for folder in folders:
        for file in os.listdir(f'{sale_folder}\\{folder}'):
            if '$' not in file:
                temp_df = pd.read_excel(os.path.join(os.getcwd(), sale_folder, folder, file), skiprows=2)
                skiper = temp_df[temp_df.isna().all(axis=1)].iloc[-1,:].name + 3
                temp_df = pd.read_excel(os.path.join(os.getcwd(), sale_folder, folder, file), skiprows=skiper, usecols='B:C')
                temp_df = temp_df.rename(columns={'Unnamed: 1': 'Интервал', 'Unnamed: 2': 'Значение'})
                temp_df = temp_df[temp_df['Интервал'] != 'Итого оборот:']
                temp_df['Интервал'] = temp_df['Интервал'].map(intervals[file[:file.find('.')]])
                temp_df = temp_df[temp_df['Интервал'] != 'Удалить']
                temp_df['Тип операции'] = folder
                temp_df = temp_df[['Интервал', 'Тип операции', 'Значение']]
                if 'Оборот' in folder:
                    temp_df['Значение'] = temp_df['Значение'] / 75
                temp = temp.append(temp_df)
    temp = temp.sort_values(by=['Интервал', 'Тип операции'], key=sort_func) 
    temp.to_excel(indicators, sheet_name=sale_folder.split('_')[-1], index=False)
    
indicators.save()
indicators.close()

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## Работаем с отчетом RFM и ВОО

In [6]:
rfm = pd.read_excel([file for file in os.listdir() if all(['rfm' in file.lower(), '$' not in file, 'holidays' not in file])][0], usecols=['ФИО клиента', 'Телефон', 'Количество операций за период', 'Сумма операций за период, (рубл. экв.)', 'Доход', 'Номер карты лояльности'], skiprows=2, sheet_name='RFM подробный')

voo = pd.read_excel([file for file in os.listdir() if all(['buyerrep' in file.lower(), '$' not in file, 'half' not in file])][0], usecols=['ФИО', 'Кол-во операций до начала периода'], skiprows=2)

voo_half_year = pd.read_excel([file for file in os.listdir() if all(['buyerrep' in file.lower(), '$' not in file, 'half' in file])][0], usecols=['ФИО'], skiprows=2)

rfm = rfm.merge(voo, how='left', left_on ='ФИО клиента', right_on='ФИО')

rfm.drop('ФИО', axis=1, inplace=True)

rfm = rfm.drop_duplicates(subset=['ФИО клиента', 'Телефон'])

rfm['Кол-во операций до начала периода'] = rfm['Кол-во операций до начала периода'].fillna(0)

rfm = rfm.merge(voo_half_year, how='left', left_on ='ФИО клиента', right_on='ФИО')

rfm = rfm.drop_duplicates(subset=['ФИО клиента', 'Телефон'])

rfm['ФИО'] = rfm['ФИО'].fillna(0)

rfm = rfm.rename(columns={'ФИО': 'Операции пол года назад'})

def client_type(row):
    
    if row['Кол-во операций до начала периода'] == 0:
        return 'Новый'
    elif row['Операции пол года назад'] == 0:
        return 'Забывший'
    else:
        return 'Постоянный'
    

rfm['Номер карты лояльности'] = rfm['Номер карты лояльности'].fillna('Нет карты')

rfm['Карта'] = rfm['Номер карты лояльности'].apply(lambda x: 0 if x == 'Нет карты' else 1)

rfm['Тип'] = rfm.apply(client_type, axis=1)

rfm = rfm.reset_index(drop=True)



## Будем считывать и записывать показатели для отчета

In [7]:
dct_indicators = {}

In [8]:
dct_indicators.setdefault('количество клиентов, предоставивших документы', rfm[(rfm['ФИО клиента'] != ' Без паспорта') & (rfm['Сумма операций за период, (рубл. экв.)'] >= 3100000)].shape[0])

108

In [9]:
nichosi_path = r'D:\FTP-DATA\Analytics\Banki_ru_nichosi'

nichosi = pd.read_excel(os.path.join(nichosi_path, sorted([(file, os.stat(f'{nichosi_path}\\{file}').st_mtime) for file in os.listdir(nichosi_path) if 'ничего себе' in file and '$' not in file], key=lambda x: x[1], reverse=True)[0][0]))

nichosi = nichosi.set_index('Unnamed: 0')

dct_indicators.setdefault('количество клиентов с документами, получивших скидку', nichosi.loc['кол-во клиентов по акции (предоставили документы)', :].iloc[0,:].dropna()[-2])

5

In [10]:
group_rfm = rfm[rfm['ФИО клиента'] != ' Без паспорта'].groupby('Тип').agg({'ФИО клиента': 'count', 'Доход': sum})

dct_indicators.setdefault('Количество новых клиентов', group_rfm.loc['Новый', 'ФИО клиента'])

dct_indicators.setdefault('Доход от новых клиентов', group_rfm.loc['Новый', 'Доход'])

dct_indicators.setdefault('Количество "стабильных" клиентов, которые совершали операции за последние 6 месяцев', group_rfm.loc['Постоянный', 'ФИО клиента'])

dct_indicators.setdefault('Доход от "стабильных" клиентов', group_rfm.loc['Постоянный', 'Доход'])

dct_indicators.setdefault('Количество "забывших" клиентов, которые не совершали операции 7 мес и более ', group_rfm.loc['Забывший', 'ФИО клиента'])

dct_indicators.setdefault('доход от "Забывших" клиентов', group_rfm.loc['Забывший', 'Доход'])

2239276.7900000005

In [11]:
rfm_card = rfm[rfm['Карта'] == 1]

dct_indicators.setdefault('Количество клиентов с картами лояльности', rfm_card.shape[0])

dct_indicators.setdefault('Количество операций с картами лояльности', rfm_card['Количество операций за период'].sum())

2033

In [12]:
cards_operation = pd.read_excel([file for file in os.listdir() if all(['sb_lp_card_op_rep' in file.lower(), '$' not in file])][0], sheet_name='для аналитики')

cards_operation['Дата совершения операции'] = pd.to_datetime(cards_operation['Дата совершения операции'], format='%d.%m.%Y')

cards_operation = cards_operation[cards_operation['Дата совершения операции'].isin(pd.date_range(dates[0], dates[1]))]

dct_indicators.setdefault('Накоплено баллов', cards_operation['Начислено баллов за операцию'].sum())

dct_indicators.setdefault('Объем операций лояльных клиентов, у.е.', cards_operation['Начислено баллов за операцию'].sum())

9305097

In [13]:
dct_indicators.setdefault('Доход от лояльных клиентов млн.руб.', rfm_card['Доход'].sum())

2521314.3499999996

In [14]:
installations = pd.read_excel([file for file in os.listdir() if all(['установки' in file.lower(), '$' not in file])][0])

report_month = Counter([element.month for element in pd.date_range(dates[0], dates[1])]).most_common()[0][0]

installations['Месяц'] = pd.to_datetime(installations['Месяц'], format='%Y_%m')

dct_indicators.setdefault('Количество скачиваний приложения', installations[installations['Месяц'].isin([datetime.date(year, report_month, 1)])]['Установок'].iloc[0])

1282

In [15]:
cards_list = pd.read_excel([file for file in os.listdir() if all(['список_карт' in file.lower(), '$' not in file])][0])

cards_list = cards_list.dropna(subset=['Дата выдачи'])

cards_list['Дата выдачи'] = pd.to_datetime(cards_list['Дата выдачи'].apply(lambda x: x[:10]), format='%d.%m.%Y')

cards_list = cards_list[cards_list['Дата выдачи'].isin(pd.date_range(dates[0], dates[1]))]

dct_indicators.setdefault('Количество новых участников клуба', cards_list.shape[0])

209

In [16]:
active_clients = pd.read_excel([file for file in os.listdir() if all(['статистика' in file.lower(), '$' not in file])][0])

dct_indicators.setdefault('Количество активных участников клуба (заходили в приложение)', active_clients[active_clients['Дата'].isin(pd.date_range(dates[0], dates[1]))]['Уникальных пользователи'].max())

1527

In [17]:
rfm_holidays = pd.read_excel([file for file in os.listdir() if all(['rfm' in file.lower(), '$' not in file, 'holidays' in file])][0], usecols=['ФИО клиента', 'Телефон', 'Количество операций за период', 'Сумма операций за период, (рубл. экв.)', 'Доход', 'Номер карты лояльности'], skiprows=2, sheet_name='RFM подробный')

rfm_holidays = rfm_holidays[(rfm_holidays['ФИО клиента'] != ' Без паспорта') & (rfm_holidays['Сумма операций за период, (рубл. экв.)'] >= 40000)]

dct_indicators.setdefault('выходные_Количество клиентов от 40 тыс', rfm_holidays.shape[0])

1668

In [18]:
pd.Series(dct_indicators).to_excel('Прочие_показатели.xlsx')